### 3. 최상위, 최하위 브랜드 인식 조사를 위한 네이버 카페 크롤링  

- 〈서울시 인터넷 쇼핑몰 100개 평가 정보〉에 따르면 2020년도 전체평가 기준 최고 점수를 받은 브랜드는 '마켓컬리'이며 최하위점수를 받은 브랜드는 '트립닷컴'이다.  
- '마켓컬리'와 '트립닷컴'에 대한 내용을 담은 네이버 카페 글(제목, 내용, 댓글)을 크롤링하여 텍스트 파일로 생성했다.  
- 네이버 카페 검색 기준 : 일반글, 작성일자(2020/01/01-2020/12/31), 관련도 순으로 정렬  
- 함수로 생성하여 검색 키워드와 몇 개의 글을 크롤링 할 것인지를 입력하도록 하였다.  

In [ ]:
def naver_cafe(keyword, num):

    # 모듈
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys
    from selenium import webdriver
    from bs4 import BeautifulSoup
    from tqdm.notebook import tqdm
    import time
    import re

    # 크롬 드라이버
    s = Service("c:/data/chromedriver.exe")
    driver = webdriver.Chrome(service = s)

    driver.get("https://search.naver.com/search.naver?where=article&query=" + keyword)
    
    time.sleep(2)
    
    # 옵션 클릭
    driver.find_element(By.XPATH, "//*[@id=\"snb\"]/div[1]/div/div[2]").click()
        
    # 일반글 선택
    driver.find_element(By.XPATH, "//*[@id=\"snb\"]/div[2]/ul/li[2]/div/div/a[3]").click()
    
    time.sleep(1)
    
    # 관련도 순으로 정렬
    driver.find_element(By.XPATH, "//*[@id=\"snb\"]/div[2]/ul/li[3]/div/div/a[1]").click()
    
    time.sleep(1)
    
    # 기간 직접 입력
    driver.find_element(By.XPATH, "//*[@id=\"snb\"]/div[2]/ul/li[4]/div/div[1]/a[9]").click()
    
    time.sleep(2)
    
    # 기간 설정 2020/01/01 - 2020/12/31
    driver.find_element(By.XPATH, "//*[@id=\"snb\"]/div[2]/ul/li[4]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[31]").click()
    driver.find_element(By.XPATH, "//*[@id=\"snb\"]/div[2]/ul/li[4]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[1]").click()
    driver.find_element(By.XPATH, "//*[@id=\"snb\"]/div[2]/ul/li[4]/div/div[2]/div[1]/span[3]/a").click()
        
    driver.find_element(By.XPATH, "//*[@id=\"snb\"]/div[2]/ul/li[4]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[31]").click()
    driver.find_element(By.XPATH, "//*[@id=\"snb\"]/div[2]/ul/li[4]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[12]").click()
    driver.find_element(By.XPATH, "//*[@id=\"snb\"]/div[2]/ul/li[4]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[31]").click()
    
    # 기간 적용
    driver.find_element(By.CLASS_NAME, "btn_apply._apply_btn").click()
    
    time.sleep(2)
    
    # 스크롤 끝까지 내리기
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    params = []
    
    while True:
        # 스크롤을 아래로 내리기
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        time.sleep(2)

        # 지금까지 스크롤 내린 현재 페이지의 높이를 가져오기
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        # 현재 페이지 높이가 이 페이지의 마지막이면 끝내기
        if new_page_height == last_page_height:
    
            html = driver.page_source
            time.sleep(2)
            
            soup = BeautifulSoup(html, 'html.parser')
            txt = soup.select("div.total_area > a")

            for i in txt:
                params.append(i.get('href'))
                
            break

        # 그렇지 않으면 마지막 높이가 될 때까지 진행하는 것
        last_page_height = new_page_height
    
    # 중복이 있는지 확인(순서 유지)
    params2 = list(dict.fromkeys(params))
        
    # 번호 세기
    cnt = 1
    
    # 텍스트 파일 생성
    f = open("c:/data/cafe_data/"+ keyword + ".txt", "w", encoding = "utf8")
    
    for i in tqdm(params2):
        driver.get(i)
        time.sleep(7)
        
        try: 
            driver.switch_to.frame("cafe_main")
            time.sleep(2)
            
            html2 = driver.page_source
            soup2 = BeautifulSoup(html2, 'html.parser')
                       
            f.write('[' + str(cnt) + ']')
            
            # 제목 크롤링
            title = soup2.select('div.title_area > h3')
            for i in title:
                f.write(re.sub('[\n\r\t]','',i.text).lstrip(' ') + '\n')
            
            # 내용 크롤링
            content = soup2.select('div.se-main-container')
            if len(content) > 0 :
                for i in content:
                    f.write(re.sub('[\n\r\t]','',i.text) + '\n')
            
            else:
                content2 = soup2.select('div.ContentRenderer')
                for i in content2:
                    f.write(re.sub('[\n\r\t]','',i.text) + '\n')  
            
            time.sleep(2)
            
            # 댓글 크롤링
            comment = soup2.select('span.text_comment')
            com_t = '' 
            for i in comment:
                if keyword in i.text:
                    com_t += re.sub('[\n\r\t]','',i.text) + ' '
                            
            time.sleep(1)
            
            f.write(com_t + '\n\n')
            
            # 게시글을 num만큼 가져오기
            cnt += 1
            if cnt > num:
                break
        
        except:
            continue
            
    f.close()

In [ ]:
naver_cafe('마켓컬리', 200)
naver_cafe('트립닷컴', 200)
# 200개의 글을 크롤링하여 텍스트 파일로 생성